In [14]:
import os
from transformers import AutoTokenizer, LlamaForCausalLM
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import glob
import re
from bs4 import BeautifulSoup
from io import StringIO

load_dotenv()

# MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=os.getenv("HF_TOKEN"))
# model = LlamaForCausalLM.from_pretrained(MODEL_NAME, token=os.getenv("HF_TOKEN"))

True

In [15]:
"""
df: dataframe
path: str path the store df
"""
def to_csv(df, path):
    return df.to_csv(path)

def to_html(df, path):
    return df.to_html(path)

def to_tsv(df, path):
    return df.to_csv(path, sep='\t')
    
"""
df_type: data frame type to be returned any of (csv, html, tsv)
task: QA task to be performed any of (arithmetic, item)
row_size: row size of the dataset
col_size: col size of the dataset
file_name: name of file
"""
def generate_dataset(df_type, task, row_size, col_size, file_name):
    columns = ['Col ' + str(i+1) for i in range(col_size)]
    rows = ['Row ' + str(i+1) for i in range(row_size)]
    if task == 'arithmetic': 
        df = pd.DataFrame(np.random.randint(1, 11, size=(row_size, col_size)), columns=columns, index=rows)
    elif task == 'item':
        df = pd.DataFrame(np.random.choice(list(string.ascii_uppercase), size=(row_size, col_size)), columns=columns, index=rows)
    df_type_dict = {'csv': to_csv, 'html': to_html, 'tsv': to_tsv}
    path = '../datasets/tables/' + file_name + '.' + df_type 
    return df_type_dict[df_type](df, path)

In [16]:
"""
generate nxn dim data
N_list: list(int) of NxN dim data to generate
"""
def generate_n_data_set(N_list):
    # 5 samples
    # Each with 3 types
    # Each with 2 tasks
    # Resulting in 30 samples
    for n in N_list:
        generate_dataset('tsv', 'arithmetic', n, n, 'arithmetic_'+str(n))
        generate_dataset('csv', 'arithmetic', n, n, 'arithmetic_'+str(n))
        generate_dataset('html', 'arithmetic', n, n, 'arithmetic_'+str(n))
    
        generate_dataset('tsv', 'item', n, n, 'item_'+str(n))
        generate_dataset('csv', 'item', n, n, 'item_'+str(n))
        generate_dataset('html', 'item', n, n, 'item_'+str(n))

In [17]:
# Arithmetic operation functions
def get_col_max(df, col_name):
    return df[col_name].max()

def get_col_min(df, col_name):
    return df[col_name].min()

def get_col_sum(df, col_name):
    return df[col_name].sum()

def get_row_max(df, row_num):
    return max(df.iloc[row_num].tolist()[1:])

def get_row_min(df, row_num):
    return min(df.iloc[row_num].values.tolist()[1:])

def get_row_sum(df, row_num):
    return sum(df.iloc[row_num].values.tolist()[1:])

# Item operation functions
def get_col_item(df, col_name):
    return ','.join(df[col_name].tolist())

def get_row_item(df, row_num):
    return ','.join(df.iloc[row_num].values.tolist()[1:])
    
"""
A qa.csv file contain all the questions and their corresponding context and answer
question: str (arithmetic operations include fine the max, min, sum)
answer: str
context: str (only the name of the table)
id: unique str
task: optional enum (“arithmetic” and “item”)
direction: optional enum (“row” or “col”)
size: optional tuple[int] 
"""
ARITHMETIC_OPERATIONS = ['maximum', 'minimum', 'sum']
DIRECTIONS = ['row', 'col']

def get_question_answer(df, task, direction):
    question = ""
    answer = ""
    if (task == 'arithmetic'):
        # randomly select an operation to perform
        operation_ind = np.random.randint(0,len(ARITHMETIC_OPERATIONS)) # 0, 1, 2
        operation = ARITHMETIC_OPERATIONS[operation_ind]
        col_arithmetic_operation_dict = {'maximum': get_col_max, 'minimum': get_col_min, 'sum': get_col_sum}
        row_arithmetic_operation_dict = {'maximum': get_row_max, 'minimum': get_row_min, 'sum': get_row_sum}
        
        if (direction == 'col'):
            col_num = np.random.randint(0, df.shape[1]-1) # col nums are added one more
            col_name = "Col %d" %(col_num+1)
            question = "What is the %s of %s?" % (operation, col_name)
            answer = col_arithmetic_operation_dict[operation](df, col_name)
            
        elif (direction == 'row'):
            row_num = np.random.randint(0, df.shape[0])   
            row_name = "Row %d" %(row_num+1)
            question = "What is the %s of %s?" % (ARITHMETIC_OPERATIONS[operation_ind], row_name)
            answer = row_arithmetic_operation_dict[operation](df, row_num)

    elif (task == 'item'):
        if (direction == 'col'):
            col_num = np.random.randint(0, df.shape[1]-1) # col nums are added one more
            col_name = "Col %d" %(col_num+1)
            question = "List all items in %s" % (col_name)
            answer = get_col_item(df, col_name)
            
        elif (direction == 'row'):
            row_num = np.random.randint(0, df.shape[0])   
            row_name = "Row %d" %(row_num+1)
            question = "List all items in %s" % (row_name)
            answer = get_row_item(df, row_num)
          
    return [question, answer]

def get_context(file_path):
    idx = file_path.find('tables')
    return file_path[idx:]

def get_id(i):
    return "nt-" + str(i)

def get_task(df):
    series = df.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())[1:] # col nums are added one more
    if (series.all()):
        return 'arithmetic'
    return 'item'

def get_direction():
    #TODO Each with two direction
    return ""

def get_size(df):
    reshaped = (df.shape[0], df.shape[1]-1)# col nums are added one more
    return str(reshaped)

def read_html(path):
    #TOOD
    table = BeautifulSoup(open(path,'r').read()).find('table')
    df = pd.read_html(StringIO(str(table)))[0]
    return df
    
def tables_to_dataset():
    path = '../datasets/tables/*'
    file_paths = glob.glob(path)
    dataset_df = pd.DataFrame(columns=['question', 'answer', 'context', 'id', 'task', 'direction', 'size'])
    i = 0
    for file_path in file_paths:
        # Read file
        if (file_path.endswith('.csv')):
            df = pd.read_csv(file_path)
        elif (file_path.endswith('.tsv')):
            df = pd.read_csv(file_path, sep='\t')
        elif (file_path.endswith('.html')):
            df = read_html(file_path)

        file_name = file_path.split('/')[-1] 
        
        # Define task
        task = get_task(df)
        # Generate q, a for all directions
        for direction in DIRECTIONS:
            q,a = get_question_answer(df, task, direction)
            dataset_df.loc[i] = [q, a, get_context(file_path), get_id(i), task, direction, get_size(df)]
            i += 1
    dataset_path = '../datasets/self_generated_qa.csv'
    return dataset_df.to_csv(dataset_path, index=False)

In [18]:
"""
split test train dev into desired percentage
train_percent: int
test_percent: int
val_percent: int
"""
def split(train_percent, test_percent, val_percent):
    dataset_path = '../datasets/self_generated_qa.csv'
    df = pd.read_csv(dataset_path)

    probs = np.random.rand(len(df))
    training_mask = probs < train_percent 
    test_mask = (probs>=train_percent) & (probs < test_percent)
    validation_mask = probs >= val_percent 
        
    train = df[training_mask]
    test = df[test_mask]
    validation = df[validation_mask]
    
    train_path = '../datasets/self_generated_train/qa.csv'
    val_path = '../datasets/self_generated_val/qa.csv'
    test_path = '../datasets/self_generated_test/qa.csv'
    
    train.to_csv(train_path, index=False)
    validate.to_csv(val_path, index=False)
    test.to_csv(test_path, index=False)

In [19]:
N = [4,6,8,10,12]
# generate n x n data
generate_n_data_set(N)

In [20]:
# generate qa
tables_to_dataset()

In [29]:
# split into train test dev
split(0.7, 0.85, 0.85) # 70% train, 15% trest, 15% val

In [283]:
# WIKI table example
# TODO: .examples?
wiki_path = '../datasets/data/*.tsv'
wiki_file_paths = glob.glob(wiki_path)

def wiki_tables_to_dataset():
    dataset_df = pd.DataFrame(columns=['question', 'answer', 'context', 'id', 'task', 'direction', 'size'])
    for i in range(len(wiki_file_paths)):
        file_path = wiki_file_paths[i]
        df = pd.read_csv(file_path, sep='\t', on_bad_lines='skip')
        dataset_df['question'] = df['utterance']
        dataset_df['answer'] = df['targetValue']
        dataset_df['context'] = df['context']#.apply(lambda x: x.split('/'[-1:]))
        dataset_df['id'] = df['id']
    dataset_path = '../datasets/wiki_qa.csv'
    return dataset_df.to_csv(dataset_path, index=False)


def merge_tables_and_wiki_tables():
    dataset_path = '../datasets/qa.csv'
    self_generated_path = '../datasets/self_generated_qa.csv'
    wiki_dataset_path = "../datasets/wiki_qa.csv"
    df =  pd.read_csv(self_generated_path)
    wiki_df =  pd.read_csv(wiki_dataset_path)
    merged_df = pd.concat([df, wiki_df], ignore_index=True)
    return merged_df.to_csv(dataset_path, index=False)

In [7]:
wiki_tables_to_dataset()

In [8]:
merge_tables_and_wiki_tables()